### Collect data from Spotify API using spotipy library

In [35]:
import datetime as dt
from dotenv import load_dotenv
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import numpy as np
import os
import pandas as pd
from pandas.io.json import json_normalize
from pathlib import Path
import re
import requests
import spotipy
import spotipy.util as util

%matplotlib inline

In [36]:
load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REDIRECT_URI = os.getenv('REDIRECT_URI')
USERNAME = os.getenv('SPOTIFY_USERNAME')
SCOPE = 'playlist-read-private'

Hierachy:
- spotipy_userauth
- create_dataset > arg(get_folder_analysis)
                 > get_playlist_analysis 
                 > get_segments > arg(extract_tracks_analysis) > arg(extract_tracks)

  - get_segments > arg(track_analysis), track_analysis_to_df, convert_time

     - track_analysis_to_df > arg(track_analysis) or > extract_track_analysis(arg(track_id))

  - extract_track_analysis > arg(tracksid), spotipy.audio_analysis
  - extract_tracks > arg(playlist_id), spotipy.playlist_tracks
   
- Using USER's playlist: get_pl_details >  playlist_id_url > arg(extract_playlists,)

   
Reduntant - tracks_analysis, track_genre

In [38]:
def spotipy_userauth2(username=USERNAME, scope=SCOPE,client_id=CLIENT_ID,\
                      client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI):
    
    '''
    Creates authorization token and returns spotipy object. util prompt does not refresh token
    and maybe deprecated in future.
    '''
    
    token = util.prompt_for_user_token(username = username, scope=SCOPE,client_id=CLIENT_ID,\
                                       client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI)
    
    sp = spotipy.Spotify(auth=token)
    
    return sp

def spotipy_userauth(username=USERNAME,scope=SCOPE,client_id=CLIENT_ID,\
                     client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI):
    
    '''
    Creates authorization token and returns spotipy object. Token automatically refreshes.
    '''
    username = USERNAME
    spotify = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=SCOPE,client_id=CLIENT_ID,\
                                       client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI))
    
    return spotify

In [39]:
def extract_playlists(spotipyUserAuth, username):
    
    '''
    Extract user's playlists' details
    
    spotipyUserAuth : spotipy object from 'spotipy_userauth' function.
    username : username(string)
    
    Returns a list of dictionary containing details of individual playlists.
    '''
    playlists = spotipyUserAuth.user_playlists(username) 
    playlistsdetails = playlists['items']
   
    return playlistsdetails 

In [40]:
def playlists_id_url(playlistsdetails):
    
    '''
    Collects and returns lists of playlist names, IDs, URLs, 
    and number of tracks present in a playlist
    
    playlistsdetails : list of dictionary containing details of individual
                       playlist. Obtained from spotipy.user_playlists.
                       
    Returns :  list of playlists' total tracks, url, and ids.
    '''
    
    pl_name = [] # Initiate playlist name list
    pl_id = []  # Initiate playlist id list
    pl_url = [] # Initiate playlist url list
    pltot_tracks = [] # Initiate playlist track count list
    
    playlistsURL = 'https://api.spotify.com/v1/playlists/'

    for i in range(len(playlistsdetails)) : 
    
        current_list = playlistsdetails[i]
        
        pl_name.append(current_list['name']) 
        pl_id.append(current_list['id'])
        url = playlistsURL + current_list['id'] + '/tracks'
        pl_url.append(url) 
        pltot_tracks.append(current_list['tracks']['total'])
    
    return pl_name, pl_id, pl_url, pltot_tracks

In [41]:
def get_pl_details(sp, username):

    pl_details = extract_playlists(sp, username)

    pl_name, pl_id, pl_url, pltot_tracks = playlists_id_url(pl_details)

    return pl_name, pl_id, pl_url, pltot_tracks

In [42]:
def filtersort_playlists(pl_name, pl_id, pl_url, pltot_tracks, key_words = None, start = 0, pl_range = 10):
    
    '''
    Filters playlists based on provided keywords present in the name of the playlists or the 
    first n playlists present in the range provided.
    Sorts ascending by total number of tracks in a playlist.
    
    playlist name, id, url , total tracks : returned from the 'playlist_id_url' function.
    Key_words = list of words (genre/terms). Default None.
    pl_range = First 'n' number of playlists. Default 10
    
    Returns: filtered and sorted list of tuples - 
             (playlist name, id, url and # of tracks).
    '''
    
    fil_pl_name = []  # Initiate filtered playlist name list
    fil_pltot_tracks = []  # Initiate filtered total tracks list
    fil_pl_id = []  # Initiate filtered playlist ID list
    fil_pl_url = []  # Initiate filtered playlist URL list
    
    if key_words is not None:
    
        for i in range(len(pl_name)):
    
            name = pl_name[i]
            if any( word in name for word in key_words):
                fil_pl_name.append(name)
                fil_pltot_tracks.append(pltot_tracks[i])
                fil_pl_id.append(pl_id[i])
                fil_pl_url.append(pl_url[i])
    else:
        
        for i in range(start, pl_range):
            
            fil_pl_name.append(pl_name[i])
            fil_pltot_tracks.append(pltot_tracks[i])
            fil_pl_id.append(pl_id[i])
            fil_pl_url.append(pl_url[i])
        
        
    sorted_pl = sorted(zip(fil_pltot_tracks, fil_pl_name, fil_pl_id, fil_pl_url), reverse = True)
    
    return sorted_pl
    

In [43]:
def extract_tracks(spotipyUserAuth, playlist_id, allCol = False, showkeys = False):
    
    '''
    Extract track info of all tracks in a playlist. 
    
    spotipyUserAuth : spotipy object from 'spotipy_userauth' function.
    playlist_id : playlist id can be obtained from  'extract_playlists'
                  or 'filtersort_playlists' function.
    allCol : Default False - Returns a dataframe with only track name and id.
             True - Returns a complete dataframe of track details 
                    with all columns.
    
    showkeys : Prints all column names/keys of the complete dataframe 
    
    Returns: Dataframe with track info (Default - name and id)
    
    '''
    track_lim = 100
    
    tracks = spotipyUserAuth.playlist_tracks(playlist_id)
    tracks_json = [tracks['items'][j]['track'] for j in range(len(tracks['items']))]
    tracks_df = json_normalize(tracks_json, sep ='_')
    
    if tracks['total'] > track_lim :
        offset = track_lim
        tracks_dflist = [tracks_df]
        
        for i in range(int(tracks['total']/track_lim)):
            
            tracks = spotipyUserAuth.playlist_tracks(playlist_id, offset = (i+1)*offset)
            tracks_json = [tracks['items'][j]['track'] for j in range(len(tracks['items']))]
            tracks_df_ = json_normalize(tracks_json, sep ='_') 
            tracks_dflist.append(tracks_df_)
    
        tracks_df = pd.concat(tracks_dflist, ignore_index = True)

    if allCol == False:
        df = tracks_df[['name', 'id']]
    else:
        df = tracks_df
        
    if showkeys == True:
        print('Info keys are :', tracks_df.columns)
    
    return df
    

In [44]:
def track_genre(spotipyUserAuth, album_ids):
    
    '''
    spotipyUserAuth : spotipy object from 'spotipy_userauth' function.
    album_ids : list of album ids. If a single album id is provided, it needs to be wrapped
                in a list.
    Returns : List of tuples of Name and Genre of albums
    '''
   
    # To remove any repeating album ids
    album_ids = list(set(album_ids))
    album_ids.remove(None)
    tot_albums = len(album_ids)
    print('total albums', tot_albums)
    # Limit of number of albums spotipy takes in its method 'albums'
    album_lim = 20
    
    if  tot_albums < album_lim :
        # Switch assignment so the next loop runs just once
        album_lim = tot_albums
        
    album_genre = []
    end_idx = 0
    
    
    for i in range(int(tot_albums/album_lim)):

        start_idx = end_idx
        #print('in loop, start at: ',start_idx)
        end_idx = start_idx + album_lim
        #print('in loop, end at: ',end_idx)
        
        album_details = spotipyUserAuth.albums(album_ids[start_idx: end_idx])['albums']
        [album_genre.append((album_details[j]['name'], album_details[j]['genres'])) \
         for j in range(album_lim) if len(album_details[j]['genres']) != 0 ]

        
    if tot_albums%album_lim != 0:
        
        album_details = spotipyUserAuth.albums(album_ids[end_idx:])['albums']
        [album_genre.append((album_details[j]['name'], album_details[j]['genres'])) \
         for j in range(len(album_ids[end_idx:])) if len(album_details[j]['genres']) != 0 ]
            
    return album_genre

In [45]:
def extract_tracks_analysis(spotipyUserAuth, tracksid, showkeys = False):
    
    '''
    spotipyUserAuth : spotipy object from 'spotipy_userauth' function.
    trackids : list of track ids.
    showkeys : Default False - prints dictionary keys
    returns : list of dictionaries containing track analysis
    '''
    tracks_analysis = [spotipyUserAuth.audio_analysis(tracksid[j]) \
                      for j in range(len(tracksid))]
    
    if showkeys == True:
        print(tracks_analysis[0].keys())
    
    return tracks_analysis

In [46]:
def track_anlaysis_to_df(trackid = None, spotipyUserAuth = None,\
                      track_analysis = None):
    
    '''
    Convert track analysis dictionaries into dateframes- beats, bars, segments and sections.
    
    trackid : Spotify track id
    spotipyUserAuth : Spotipy auth object. Required if using track id
    track_analysis : Track analysis dictionary of a single track if trackid is not provided
    
    Returns : track overview (dictionary) and dataframes of beats, 
              bars, segments and sections
    '''
    
    if trackid is not None:
        if spotipyUserAuth is None:
            raise TypeError('Need spotipy authorized object')
            
        track_analysis = extract_tracks_analysis(spotipyUserAuth, [trackid])[0]
    
    
    trackoverview = track_analysis['track']
    # We don't need tatums currently
    
    beats_df = json_normalize(track_analysis['beats'], sep ='_')
    bars_df =  json_normalize(track_analysis['bars'], sep ='_')
    segments_df = json_normalize(track_analysis['segments'], sep ='_')
    sections_df = json_normalize(track_analysis['sections'], sep ='_')
    
    return trackoverview, beats_df, bars_df, segments_df, sections_df

In [47]:
def convert_time(secs):
    ''' COnverts seconds to mins. Format mm:ss 
    '''
    if pd.isna(secs):
        return float('NaN')
    else:
        int_secs = int(secs)
        if int_secs is not 0:
            milisecs = int(round(secs % int_secs,2) * 100)
        else:
            milisecs = int(round(secs,2) * 100)

        minutes = int(int_secs / 60)
        seconds = int_secs % 60

        return '{:0>2d}:{:0>2d}:{:0>2d}'.format(minutes,seconds,milisecs)

In [48]:
'''
def tracks_analysis(spotipyUserAuth, playlist_id):
    
    
    spotipyUserAuth : Spotipy auth object.
    playlist_id : playlist id  
    *user functions extract_tracks and extract_track_analysis used here.
    
    Returns : a list of tuples : (name of the track (string), tuple containing trackoverview (dictionary), 
                            beats_df, bars_df, segments_df, sections_df)

    # extract_tracks returns a dataframe
    tracks_df = extract_tracks(spotipyUserAuth, playlist_id)
    tracks_name = list(tracks_df['name'])
    tracks_id = list(tracks_df['id'])
    #track_analysis returns a list of dictionary
    tracks_analysis_ = extract_tracks_analysis(spotipyUserAuth, tracks_id)
    analysis_dict = {}
    
    for name_, track_analysis in zip(tracks_name, tracks_analysis_):
        #trackanalysis = track_analysis_to_df(track_analysis = track_analysis)
        analysis_dict[name_] = track_analysis
        
    return analysis_dict
'''

In [49]:
def get_segments(track_analysis, segments=True, min_conf=0.5, min_dur=0.25, tempo=True, \
                 sections=False, beats=False, bars=False):
    '''
    Get segments of tracks on a playlist with conditions on  minimum confidence 
    and minimum duration of a segment. Since we are currently interested in tempo
    of a track we will be returning that value as well.
    
    trackanalysis: track analysis (dict) of a track (obtained from tracks_analysis dict)
    segments: Default True. False if segments dataframe is not needed
    min_conf: minimum confidence to include a segment (range 0-1)
    min_dur : minimum duration/length in secs to include a segment
    tempo: Default True. False if tempo value is not needed
    sections: Default False. True if sections dataframe needs to be returned
    beats: Default False. True if beats dataframe needs to be returned
    bars: Default False. True if bars dataframe needs to be returned
    
    Returns: (in this order) tempo and segments dataframe (sections_df, beats_df, bars_df  as asked)
              of a single track
    '''
    
    trackoverview, beats_df, bars_df, segments_df, sections_df = \
        track_anlaysis_to_df(track_analysis = track_analysis)
    
    if tempo:
        tempo_df = pd.DataFrame({ 'tempo' : [trackoverview['tempo']]})
    
    # Introducing start_minute column for more readability of start time in min:sec format
    start_minute = segments_df['start'].map(convert_time)
    segments_df.insert(1,'start_minute', start_minute)
    segments_df_ = segments_df[(segments_df['confidence'] > min_conf) & (segments_df['duration'] > min_dur)]
    
    while len(segments_df_) < 100:
        min_conf = min_conf - 0.05
        min_dur = min_dur - 0.05
        segments_df_ = segments_df[(segments_df['confidence'] > min_conf) & (segments_df['duration'] > min_dur)]
    
    segments_df_ = segments_df_[['start', 'start_minute', 'duration', 'confidence', 'pitches', 'timbre']]
    
    # iterating over a boolean mask to collect what to output/return
    output = [b for a, b in zip(\
           [tempo, segments, sections, beats, bars],[tempo_df, segments_df_, sections_df, beats_df, bars_df])\
           if  a]
    
    return output

In [50]:
def get_playlist_analysis(spotipyUserAuth, playlist_id, segments=True, min_conf=0.5, \
                          min_dur=0.25, tempo=True, sections=False, beats=False, bars=False):
    
    '''
    spotipyUserAuth : Spotipy auth object.
    playlist_id : playlist id
    segments and tempo: Default True. False if not needed
    min_conf: minimum confidence to include a segment (range 0-1)
    min_dur : minimum duration/length in secs to include a segment
    sections/beats/bars: Default False. True if needs to be returned
    
    Returns : a dict with key/value pairs for all tracks in the playlist 
                Keys: name of track
                Value: list containing tempo and segment dataframe of the track 
                       (and sections/beats/bars if asked)
    '''
    
    tracks_df = extract_tracks(spotipyUserAuth, playlist_id)
    tracks_name = list(tracks_df['name'])
    tracks_id = list(tracks_df['id'])
    #track_analysis returns a list of dictionary
    tracks_analysis = extract_tracks_analysis(spotipyUserAuth, tracks_id)
    playlist_analysis = {}
    
    for name_, track_analysis in zip(tracks_name, tracks_analysis):
        
        # remove any special characters from name (they may cause issues in filenaming)
        name_ = re.sub(r'[*|><:"?/]|\\',"", name_)
        playlist_analysis[name_] = get_segments(track_analysis, segments=segments, \
                                min_conf=min_conf, min_dur=min_dur, tempo=tempo, \
                                sections=sections, beats=beats, bars=bars)
    return playlist_analysis

In [51]:
def get_folder_analysis(spotipyUserAuth, filsort_pl=None, pl_name_id=None, segments=True, min_conf=0.5, \
                          min_dur=0.25, tempo=True, sections=False, beats=False, bars=False):
    
    '''
    Here, we will be using filtered and sorted output. Future edit should take user 
    playlist names and id.
    
    spotipyUserAuth : Spotipy auth object.
    
    filsort_pl : Default None. Uses 4-tuple output from filtersort_playlist function.
    pl_name_id : Dafault None. In the case filsort_pl is not available,
                 provide list of playlist name and id tuples
    
    segments and tempo: Default True. False if not needed
    min_conf: minimum confidence to include a segment (range 0-1)
    min_dur : minimum duration/length in secs to include a segment
    sections/beats/bars: Default False. True if needs to be returned
    
    Returns: a dict with key/value pairs for all playlists in the folder.
             Key : Name of the playlist (string)
             Value : a dict of track analysis of all tracks from the playlist 
             (Values are returned from get_playlist_analysis)
    '''
    
    folder_analysis = {}
    
    if filsort_pl is not None:
        for p in filsort_pl:

            # remove any special characters from name (they may cause issues in filenaming)
            pl_name = re.sub(r'[*|><:"?/]|\\',"", p[1])
            folder_analysis[pl_name] = get_playlist_analysis(spotipyUserAuth, playlist_id=p[2],\
                                                        segments=segments, tempo=tempo,\
                                                        min_conf=min_conf, min_dur=min_dur,\
                                                        sections=sections, beats=beats, bars=bars)
    else:
        for p in pl_name_id:

            # remove any special characters from name (they may cause issues in filenaming)
            pl_name = re.sub(r'[*|><:"?/]|\\',"", p[0])
            folder_analysis[pl_name] = get_playlist_analysis(spotipyUserAuth, playlist_id=p[1],\
                                                        segments=segments, tempo=tempo,\
                                                        min_conf=min_conf, min_dur=min_dur,\
                                                        sections=sections, beats=beats, bars=bars)
        
    return folder_analysis

In [52]:
sp = spotipy_userauth(USERNAME)

In [53]:
pl_Name_, pl_ID_, pl_URL_, pltot_Tracks_ = get_pl_details(USERNAME)

print(pl_Name_[:23])
print(pltot_Tracks_[:])

["Today's Top Hits", 'Chill Beats Weekly 🦔 groove, relax, study (chill lofi jazzhop)', 'Deeper House', 'Deep house', 'Progressive House', 'Mo House lo Trance', 'Our old school trance 138', 'Our old school trance 3', 'Our old school trance 2', 'Our old school trance', 'That familiar trance', 'Deep Trance', 'Classic progressive', 'Housy beats', 'TBD', 'Chill out', 'Progressive 8', 'Progressive 7', 'Progressive 6', 'Progressive 5.2', 'Progressive 5', 'Progressive 3', 'Progressive 2']
[50, 139, 11, 16, 15, 42, 14, 20, 8, 27, 61, 4, 6, 3, 70, 11, 18, 8, 28, 18, 19, 8, 6, 14, 84, 6, 155, 4, 124, 1, 2, 14, 60, 21, 18, 141, 128, 85, 31, 32, 1, 20, 2, 18, 4, 15, 4, 8, 15, 65]


In [54]:
key_words = ['Progressive']
filsort_pl = filtersort_playlists(pl_Name_, pl_ID_, pl_URL_, pltot_Tracks_, start = 1, pl_range = 23)
filsort_pl

[(139,
  'Chill Beats Weekly 🦔 groove, relax, study (chill lofi jazzhop)',
  '2rN3mSrzUcgjlj1TcEDTX7',
  'https://api.spotify.com/v1/playlists/2rN3mSrzUcgjlj1TcEDTX7/tracks'),
 (70,
  'TBD',
  '2YjqtjD98B6fGX55JBzybN',
  'https://api.spotify.com/v1/playlists/2YjqtjD98B6fGX55JBzybN/tracks'),
 (61,
  'That familiar trance',
  '3PH2J5HkKhhMoxWj3W0jk8',
  'https://api.spotify.com/v1/playlists/3PH2J5HkKhhMoxWj3W0jk8/tracks'),
 (42,
  'Mo House lo Trance',
  '2RnR5cw9kJUc9onu4WSRrW',
  'https://api.spotify.com/v1/playlists/2RnR5cw9kJUc9onu4WSRrW/tracks'),
 (28,
  'Progressive 6',
  '53kl8WegufR0IqMgyklXEL',
  'https://api.spotify.com/v1/playlists/53kl8WegufR0IqMgyklXEL/tracks'),
 (27,
  'Our old school trance',
  '7HUclibRF0h57pVvXr3g9v',
  'https://api.spotify.com/v1/playlists/7HUclibRF0h57pVvXr3g9v/tracks'),
 (20,
  'Our old school trance 3',
  '43sMbiw98RFTH2WjmbTidr',
  'https://api.spotify.com/v1/playlists/43sMbiw98RFTH2WjmbTidr/tracks'),
 (19,
  'Progressive 5',
  '2IlSKDObaH9jwLJQcqiv

In [55]:
#folder_analysis_dict = get_folder_analysis(sp, filsort_pl[2:-2])

In [56]:
#folder_analysis_dict.keys()

In [57]:
#folder_analysis_dict['That familiar trance'].keys()

In [58]:
#folder_analysis_dict['That familiar trance']['Kunai - Radio Edit'][1]

In [59]:
def create_dataset(folder_analysis):
    '''
    Creates folders for each playlist, subfolders for all tracks in a playlist folder
    and track analysis dataframes as parquet files
    '''
    
    # Path to 'Dataset' dir
    p = Path.cwd().parent.joinpath('Dataset')
    # list of dataframe names in output 
    df_names = ['tempo', 'segments', 'sections', 'beats', 'bars']
    
    for fn, i in folder_analysis.items():
        
        path_ = p.joinpath('{}'.format(fn))
        path_.mkdir(exist_ok=True) 
        
        for track, j in i.items():
            
            for k in range(len(j)):
            
                j[k].to_parquet(path_.joinpath('{}_{}.parquet'.format(track,df_names[k])), engine = 'pyarrow')
        

In [60]:
#create_dataset(folder_analysis_dict)

In [61]:
"""
dict_sampl_ =list(folder_analysis_dict.items())[-2:]
dict_sample ={ dict_sampl_[0][0] : dict_sampl_[0][1], dict_sampl_[1][0] : dict_sampl_[1][1] }
dict_sample['Classic progressive']['Skylarking - Original Mix'][0]
"""

"\ndict_sampl_ =list(folder_analysis_dict.items())[-2:]\ndict_sample ={ dict_sampl_[0][0] : dict_sampl_[0][1], dict_sampl_[1][0] : dict_sampl_[1][1] }\ndict_sample['Classic progressive']['Skylarking - Original Mix'][0]\n"

In [62]:
def uri_to_id(uri_list):
    '''
    Parses ID from playlist URI and returns list of playlists' ID.
    Playlist IDs get fed into extract_tracks or tracks_analysis
    
    URI_list : list of playlist URIs
    returns : playlists' ID list
    '''
    id_list = [uri_list[i].split(':')[-1] for i in range(len(uri_list))]
    
    return id_list

In [63]:
URI_list = ['spotify:playlist:37i9dQZF1E3adH2woxCiuA', 'spotify:playlist:37i9dQZF1E39zk9HFlqqXb', \
            'spotify:playlist:37i9dQZF1E35s2CYBuY9kz','spotify:playlist:37i9dQZF1DXbtYAdenGE9U',\
            'spotify:playlist:37i9dQZF1DWTiVLKoHQ1yC']
x = uri_to_id(URI_list)
x

['37i9dQZF1E3adH2woxCiuA', '37i9dQZF1E39zk9HFlqqXb', '37i9dQZF1E35s2CYBuY9kz']

In [64]:
pl_nameid = [(str(i), x[i]) for i in range(len(x))]
pl_nameid

[('0', '37i9dQZF1E3adH2woxCiuA'),
 ('1', '37i9dQZF1E39zk9HFlqqXb'),
 ('2', '37i9dQZF1E35s2CYBuY9kz')]

In [65]:
folder_analysis_dict2 = get_folder_analysis(sp, pl_name_id=pl_nameid)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [70]:
folder_analysis_dict2['1'].keys()

dict_keys(['Just As You Are', 'Northern Soul', 'Lounging By The Sea - Album Mix', 'Empire Of Hearts', 'Overtones - Extended Mix', 'R We Ever Gonna Be - Original Mix', 'Am I On Your Mind - Original Mix', 'Borrow Love - Original Mix', 'CDMX', 'Hero', 'Fractal', 'Rotunda - Official Radio Edit', 'Duality', 'Lost Signal', 'Kick Off (ASOT 954)', 'Sci-Fi', 'After The Rain - Club Mix', 'Black Hole', 'Mordecai', 'It Could Be', 'Lost - Chillout Mix', 'Let The Light Shine In - Crusy & Jose de Mara Remix', 'The Sense Of Ease - Original Mix', 'Golden Sky - Sons of Maria Remix', 'Feels Good (feat. HanLei)', '4 Elements', 'Sinner', 'North Star', 'Playing With Fire - C-Systems Acoustic Rework', 'People Come, People Go - Chillout Mix', 'Burned With Desire - Rising Star Vocal Mix', 'The Air I Breathe', 'Jetlag - Ben Gold & Allen Watts Remix', 'White Sand - Chillout Mix', "J'ai Envie De Toi", 'Celeste (ABGT266)', 'The Phuture (Mixed)', 'Ave Maria', 'Need You', 'A Perfect Day in Tulum', 'Rebirth - Ahmed R

In [ ]:
folder_analysis_dict2['1']['Just As You Are']